In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
# --- Import TfidfVectorizer instead of CountVectorizer ---
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# --- Téléchargement des ressources NLTK nécessaires (si pas déjà fait) ---
# Vous pouvez exécuter ces lignes une fois dans votre environnement
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
# nltk.download('omw-1.4') # Open Multilingual Wordnet
# nltk.download('words') # Pour la liste de mots anglais

# --- Définition des Fonctions Auxiliaires ---

# Ensemble de mots anglais valides (peut être chargé depuis une source externe si nécessaire)
try:
    english_words = set(nltk.corpus.words.words())
except LookupError:
    print("NLTK 'words' corpus non trouvé. Téléchargez-le avec nltk.download('words')")
    print("Utilisation d'un ensemble vide pour english_words pour continuer l'exemple.")
    english_words = set() # Placeholder si non téléchargé

def get_wordnet_pos(treebank_tag):
    """
    Convertit les étiquettes de parties du discours (Part of Speech, POS) de Penn Treebank en format WordNet.
    """
    if treebank_tag.startswith("J"):
        return wordnet.ADJ
    elif treebank_tag.startswith("V"):
        return wordnet.VERB
    elif treebank_tag.startswith("N"):
        return wordnet.NOUN
    elif treebank_tag.startswith("R"):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Par défaut, c'est un nom

def preprocess_text(text, english_words_set=english_words, min_len_word=3, rejoin=False):
    """
    Pré-traite un texte : minuscules, suppression non-alpha, stop words, lemmatisation, mots courts.
    """
    if not isinstance(text, str): # Vérification de type simple
        return "" if rejoin else []

    text = text.lower()
    text = re.sub("[^a-z]", " ", text)
    try:
        stop_words = set(stopwords.words("english"))
    except LookupError:
        print("NLTK 'stopwords' non trouvé. Téléchargez-le avec nltk.download('stopwords')")
        stop_words = set() # Placeholder

    words = [word for word in text.split() if word not in stop_words]
    # Utilise l'ensemble de mots passé en argument
    filtered_corpus = [word for word in words if word in english_words_set]

    lemmatizer = WordNetLemmatizer()
    try:
        tagged_words = pos_tag(filtered_corpus)
    except LookupError:
         print("NLTK 'averaged_perceptron_tagger' non trouvé. Téléchargez-le.")
         # Sans POS tagging, on lemmatise comme des noms par défaut
         tagged_words = [(word, 'N') for word in filtered_corpus]

    words = [
        lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in tagged_words
    ]

    final_tokens = [w for w in words if len(w) >= min_len_word] # Corrected: >= instead of &gt;=

    if rejoin:
        return " ".join(final_tokens)

    return final_tokens

# Note: clean_dataset est redondant si on utilise TextPreprocessor avec rejoin=True

def topic_modeling_lda(reviews, num_topics):
    """
    Vectorise (TfidfVectorizer) et crée le modèle LDA. Retourne modèle, vectorizer, vecteurs de sujet, matrice TF-IDF.
    *** ATTENTION : LDA est généralement conçu pour fonctionner avec des comptes de mots (CountVectorizer).
    *** L'utilisation de TF-IDF comme entrée pour LDA est moins courante et peut donner des résultats
    *** moins interprétables car le modèle probabiliste sous-jacent de LDA est basé sur des comptes.
    """
    print("--- Attention : Utilisation de TfidfVectorizer avec LDA ---")
    # --- Utilisation de TfidfVectorizer ---
    vectorizer = TfidfVectorizer(
        max_df=0.95, min_df=1, max_features=1000, stop_words="english"
        # Attention: stop_words='english' ici peut être redondant si preprocess_text les a déjà enlevés.
        # Si preprocess_text les enlève, mettez stop_words=None ici.
    )
    # --- La matrice résultante est une matrice TF-IDF ---
    tfidf_matrix = vectorizer.fit_transform(reviews)

    lda_model = LatentDirichletAllocation(
        n_components=num_topics, learning_method="online", random_state=0, max_iter=10 # max_iter peut nécessiter ajustement
    )

    # Gérer le cas où la matrice tf-idf est vide après vectorisation
    if tfidf_matrix.shape[0] == 0 or tfidf_matrix.shape[1] == 0:
         print("Attention : La matrice TF-IDF est vide après vectorisation. LDA ne peut pas être entraîné.")
         # Retourner des valeurs par défaut ou lever une exception
         return None, vectorizer, np.array([]), tfidf_matrix

    # --- Entraînement de LDA sur la matrice TF-IDF (non standard) ---
    lda_model.fit(tfidf_matrix)
    topic_vectors = lda_model.transform(tfidf_matrix)

    # --- Retourne la matrice TF-IDF au lieu de la matrice de comptage ---
    return lda_model, vectorizer, topic_vectors, tfidf_matrix

# --- Définition des Classes Transformer pour le Pipeline ---

class TextPreprocessor(BaseEstimator, TransformerMixin):
    """
    Transformateur personnalisé pour le prétraitement de texte dans un pipeline sklearn.
    """
    def __init__(self, english_words_set, min_len_word=3, rejoin=True):
        self.english_words_set = english_words_set
        self.min_len_word = min_len_word
        self.rejoin = rejoin # Important: doit être True pour que la sortie soit une chaîne pour TfidfVectorizer

    def fit(self, X, y=None):
        # Ce préprocesseur n'a pas besoin d'apprendre quoi que ce soit du set d'entraînement
        return self

    def transform(self, X):
        # X est attendu comme un itérable de documents (ex: liste de strings, pd.Series)
        return [
            preprocess_text(doc, self.english_words_set, self.min_len_word, self.rejoin)
            for doc in X
        ]

class TopicModeler(BaseEstimator, TransformerMixin):
    """
    Transformateur personnalisé pour la modélisation de sujet LDA dans un pipeline sklearn.
    Utilise maintenant TfidfVectorizer en interne.
    """
    def __init__(self, num_topics=4):
        self.num_topics = num_topics
        self.lda_model = None
        # --- Renommer pour refléter l'utilisation de TF-IDF ---
        self.tfidf_vectorizer = None
        self.topic_vectors = None
        # --- Renommer pour refléter la matrice TF-IDF ---
        self.dtm = None # Document Term Matrix (même si c'est TF-IDF ici)

    def fit(self, X, y=None):
        # X est attendu comme une liste de textes *prétraités* (sortie de TextPreprocessor)
        result = topic_modeling_lda(X, self.num_topics)
        if result[0] is not None: # Vérifier si LDA a pu être entraîné
            # --- Stocker le TfidfVectorizer et la matrice TF-IDF ---
            self.lda_model, self.tfidf_vectorizer, self.topic_vectors, self.dtm = result
        else:
            # Gérer le cas où l'entraînement a échoué (matrice vide)
            print("Échec de l'entraînement du TopicModeler.")
            self.lda_model = None
            self.tfidf_vectorizer = result[1] # Le vectorizer est quand même créé
            self.topic_vectors = np.array([])
            self.dtm = result[3]
        return self

    def transform(self, X):
        # X est attendu comme une liste de textes *prétraités* (sortie de TextPreprocessor)
        # Important: Utilise le vectorizer et le modèle LDA *déjà entraînés* (fittés)
        # --- Vérifier si le TfidfVectorizer est entraîné ---
        if self.lda_model is None or self.tfidf_vectorizer is None:
             print("Erreur: Le TopicModeler n'a pas été entraîné correctement.")
             num_samples = len(X) if isinstance(X, list) else X.shape[0]
             return np.zeros((num_samples, self.num_topics))

        # 1. Vectoriser les nouvelles données avec le TfidfVectorizer entraîné
        tfidf_matrix_new = self.tfidf_vectorizer.transform(X)
        # 2. Obtenir la distribution de topics avec le modèle LDA entraîné
        #    (Note: transforme basé sur la matrice TF-IDF, ce qui est non standard pour LDA)
        topic_distribution = self.lda_model.transform(tfidf_matrix_new)
        return topic_distribution

# --- Création et Utilisation du Pipeline ---

# Définir le nombre de topics
num_topics = 4

# Créer l'instance du pipeline
text_pipeline = Pipeline(
    [
        (
            "preprocessing",
            TextPreprocessor(english_words_set=english_words, min_len_word=3, rejoin=True),
        ),
        # La sortie de 'preprocessing' est une liste de strings nettoyées
        # Cette sortie est l'entrée pour 'topic_modeling'
        # 'topic_modeling' utilise maintenant TfidfVectorizer en interne
        ("topic_modeling", TopicModeler(num_topics=num_topics))
        # La sortie de 'topic_modeling' est la distribution de probabilité sur les topics
    ]
)

# --- Exemple d'utilisation ---

# 1. Entraîner le pipeline sur un corpus complet
corpus_complet = [
    "The customer service was terrible and slow, really bad experience.",
    "Room was clean but the bed was uncomfortable.",
    "Breakfast lacked variety and coffee was cold.",
    "Staff were very friendly and helpful during check-in.",
    "Great location near the city center and attractions.",
    "Food at the hotel restaurant was excellent, especially the pasta.",
    "Bathroom wasn't very clean, found hair in the shower.",
    "Hotel was hard to find, poor signage and location.",
    "Very noisy room facing the street, couldn't sleep well.",
    "Quick check out process, efficient staff."
]

print("Entraînement du pipeline complet...")
try:
    text_pipeline.fit(corpus_complet)
    print("Entraînement terminé.")
except Exception as e:
    print(f"Erreur pendant l'entraînement du pipeline : {e}")
    # Arrêter si l'entraînement échoue - Peut-être ajouter un sys.exit(1) ou autre gestion d'erreur

# 2. Utiliser le pipeline entraîné pour prédire le topic d'un nouveau texte
texte_unique_raw = "The check-in staff took forever and seemed unfriendly."
review0_iterable = [texte_unique_raw] # Doit être un itérable (liste ou Series)

print(f"\nPrédiction du topic pour le texte : '{texte_unique_raw}'")

try:
    # Obtenir la distribution de probabilité des topics pour le texte unique
    topic_distribution = text_pipeline.transform(review0_iterable)

    # Trouver le numéro du topic le plus probable (index du max)
    numero_topic_predit = np.argmax(topic_distribution, axis=1)[0]

    print(f"Distribution de probabilité : {topic_distribution[0]}")
    print(f"Le numéro du topic prédit (0 à {num_topics-1}) est : {numero_topic_predit}")

    # Optionnel : Afficher les mots clés pour interpréter les topics (si l'entraînement a réussi)
    if hasattr(text_pipeline.named_steps['topic_modeling'], 'lda_model') and \
       text_pipeline.named_steps['topic_modeling'].lda_model is not None:
        print("\n--- Mots clés par topic (appris lors du fit) ---")
        # --- Accéder au TfidfVectorizer ---
        vectorizer = text_pipeline.named_steps['topic_modeling'].tfidf_vectorizer
        lda_model = text_pipeline.named_steps['topic_modeling'].lda_model
        feature_names = vectorizer.get_feature_names_out()
        n_top_words = 7
        for topic_idx, topic_loadings in enumerate(lda_model.components_):
            top_words_indices = topic_loadings.argsort()[:-n_top_words - 1:-1]
            top_words = [feature_names[i] for i in top_words_indices]
            print(f"Topic {topic_idx}: {', '.join(top_words)}")
    else:
        print("\nImpossible d'afficher les mots clés (modèle non entraîné correctement).")

except Exception as e:
    print(f"Erreur pendant la transformation/prédiction : {e}")



Entraînement du pipeline complet...
--- Attention : Utilisation de TfidfVectorizer avec LDA ---
Entraînement terminé.

Prédiction du topic pour le texte : 'The check-in staff took forever and seemed unfriendly.'
Distribution de probabilité : [0.10490197 0.11182256 0.10501788 0.6782576 ]
Le numéro du topic prédit (0 à 3) est : 3

--- Mots clés par topic (appris lors du fit) ---
Topic 0: bed, uncomfortable, coffee, variety, cold, breakfast, room
Topic 1: restaurant, quick, food, efficient, process, especially, excellent
Topic 2: bathroom, hard, poor, shower, clean, hair, face
Topic 3: friendly, helpful, staff, great, near, center, city


c:\Users\emman\Desktop\open\openpy3.12.9\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
